## Lógica para a execução do código:
1. Listar todos os carros que rodaram no dia &#10003;
2. Listar as linhas que estes carros rodaram &#10003;
3. Listar as rotas ociosas de cada linha &#10003;
4. Listar as rotas programadas de cada linha &#10003;
5. Listar a rota realizada pelo carro (dados mix) &#10003;
6. Fazer o comparativo entre realizada e programada &#9881;
7. Obter acesso SMTP para envio de email &#10003;
8. Enviar email com relatório (carros que foram ofensores) &#8987;
## Observações
> a). Todos os dados estão momentaneamente hospedados localmente em banco PostgreSQL.
> b). O Datalake oficial do Grupo HP ainda está em desenvolvimento, caso queiram evoluir no consumo e na analise destes dados, precisamos decidir como disponibilizar estes dados, há alguns caminhos:
>   1. Disponibilizar os dados consolidados localmente via rede.
> 2. O cliente temporariamente usar algum serviço de alguma cloud, sugestão: Google ou AWS

### * *Salvar em banco de dados usado em produção não é uma opção válida*

In [1]:
from xml.etree import ElementTree as ET
import folium
import numpy as np
from shapely.geometry import Point, LineString
import pandas as pd
from sqlalchemy import create_engine
from math import radians, cos, sin, sqrt, atan2
import ast
import json

### Autênticação

In [2]:
dbname = 'poc-db'
user = 'adminpoc'
password = 'adminpoc'
host = 'localhost'
port = '5440' 

### Variáveis

In [3]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [4]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

In [5]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

### SQL's

In [72]:
# sql_aggr_op = """ 
# SELECT execution_date, linha, carro, re, nome, direcao, idle_route, num_linha, descricao, mr_execution_date, mr_carro, made_trip_standardized
# FROM hp.aggr_operation_day;
# """

In [73]:
# sql_schedule_services = """
# SELECT id, execution_date, linha, carro, re, nome, dthr_saida, dthr_retorno, dthr_chegada
# FROM hp.schedule_operation;
# """

### Functions

In [74]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [75]:
def add_placemarks_to_map(placemarks, map):
    """ 
    param: placemarks: lista de dicionários com as informações dos placemarks
    return: map: objeto do tipo folium.Map

    Quando o arquivo conter somente um ponto, por exemplo uma tupla de coordenadas,ele será plotado como um marcador de viagem realizada. Quando houver
    uma lista de coordenadas, oriundas de um arquivo com trajetória, será plotado uma linha entre os pontos.
    """
    for i, placemark in enumerate(placemarks):
        parsed_coords = parse_coordinates(placemark['coordinates'])
        if len(parsed_coords) == 1:  # Para um único ponto
            folium.Marker(
                location=parsed_coords[0],
                icon=folium.Icon(icon='fa-van-shuttle', prefix='fa',color='blue')
            ).add_to(map)

        else:  # Para múltiplos pontos (trajetória)
            if 'VOLTA' in placemark['name']:
                folium.PolyLine(locations=parsed_coords, color='green', weight=2.5, opacity=1, dash_array='5, 5').add_to(map)

            else:  # Para outros casos, usa uma linha contínua
                folium.PolyLine(locations=parsed_coords, color='red', weight=2.5, opacity=1).add_to(map)

In [76]:
def haversine(lon1, lat1, lon2, lat2):
    # Raio da Terra em quilômetros
    R = 6371.0

    # Conversão de coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Diferenças nas coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [77]:
def parse_coordinates(coord_string):
    # Dividir a string por espaços para separar cada coordenada
    coords = coord_string.strip().split(' ')
    # Criar uma lista para armazenar as tuplas de coordenadas
    coord_list = []
    # Processar cada par de coordenadas
    for coord in coords:
        if coord:
            # Dividir cada par por vírgula para obter os valores de latitude e longitude
            lat, lng, _ = coord.split(',')
            # Converter os valores de string para float e adicioná-los como uma tupla à lista
            coord_list.append((float(lng), float(lat)))  # Invertido para corresponder a (latitude, longitude)
    return coord_list

In [78]:
def correct_coordinates(coords_list):
    # Inverte cada par de coordenadas na lista
    return [[lat, lon] for [lon, lat] in coords_list]

### Datasets

#### 1. dados da escala

In [79]:
# df_schedules_operation = pd.read_sql_query(sql_schedule_services, engine)

#### 2. dados planejados

In [80]:
# df_aggr_op = pd.read_sql_query(sql_aggr_op, engine)

In [81]:
# df_aggr_op['direcao'] = df_aggr_op['direcao'].str.upper()

#### 3. dados realizados

In [82]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)

In [83]:
df_planned_trip = pd.DataFrame(placemarks_planned)

In [84]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])

In [85]:
df_planned_trip['identificacao'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[1])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [86]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])
df_planned_trip['Descrição e Sentido'] = df_planned_trip['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip['Descrição'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_planned_trip['Sentido'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip.drop('Descrição e Sentido', axis=1, inplace=True)

In [87]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [88]:
df_planned_trip['sublinha'] = df_planned_trip['identificacao'].apply(lambda x: de_para[x[:-1]])

In [89]:
df_planned_trip['num_linha'] = df_planned_trip['Código'].astype(str) + df_planned_trip['sublinha']

#### 4. Dados filtrados do dia

In [90]:
df_aggr_op['execution_date'] = pd.to_datetime(df_aggr_op['execution_date'])


In [91]:
df_filtered = df_aggr_op[['execution_date','linha','carro','re','nome','direcao','idle_route','made_trip_standardized']]

In [92]:
df_filtered = df_aggr_op[df_aggr_op['execution_date'] == pd.Timestamp('2024-04-15')]

In [93]:
df_report = pd.merge(df_filtered, df_planned_trip, left_on=['linha','direcao'], right_on=['num_linha','Sentido'] ,how='left')

In [94]:
df_report.drop(columns=['num_linha_y','sublinha','Sentido','Descrição','identificacao','Código','name','mr_carro','mr_execution_date','mr_carro','num_linha_x'], inplace=True)

Variáveis para listar todos os veículos

In [95]:
# list_car_day = df_filtered['carro'].unique().tolist()

Variável para realizar testes atômicos

In [96]:
list_car_day = [20018]

Criação do objeto de mapa

In [97]:
mapa = folium.Map(location=[-16.67, -49.25], zoom_start=12)

### Engine de analise de rota

In [98]:
for car in list_car_day:
    #print(car) 
    df = df_report[df_report['carro'] == car]
    row = df.iloc[0]
    if isinstance(row['made_trip_standardized'], str):
        coords = ast.literal_eval(row['made_trip_standardized'])
    else:
        coords = row['made_trip_standardized']
    polyline_coords = [(coord['lng'], coord['lat']) for coord in coords]
    folium.PolyLine(locations=polyline_coords,color="red",weight=2.5,opacity=1).add_to(mapa)
    for index, row in df.iterrows():
        # print(row['linha'])
        coords = parse_coordinates(row['coordinates'])
        folium.PolyLine(locations=coords, color="green", weight=2.5, opacity=1, popup=f"Trip {index}").add_to(mapa)
    for index, row in df.iterrows():
        coords = row['idle_route']
        polyline_coords = [(coord['lat'], coord['lng']) for coord in coords]
        folium.PolyLine(locations=polyline_coords, color="blue", weight=2.5, opacity=1, popup=f"Trip {index}").add_to(mapa)

# mapa

In [99]:
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 400px; height: 150px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;
     ">&nbsp; <b>Legenda do Mapa</b> <br>
     &nbsp; Vermelho : Rota Realizada Mix <br>
     &nbsp; Verde : Rota Programada (Papeleta) <br>
     &nbsp; Azul : Rota Ociosa <br>
      </div> 
     '''
mapa.get_root().html.add_child(folium.Element(legend_html))

#### Plotando mapa para visualização

In [100]:
mapa

#### Salvando o mapa

In [101]:
mapa.save('mapa.html')

Teste